In [3]:
#REFS:
#Short presentations for alternating and symmetric groups, J.N. Bray, M.D.E. Conder, C.R. Leedham-Green, and E.A. O’Brien
#https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Generalization_to_arbitrary_rings

In [4]:
#we can represent the group algebra of a cyclic group as a polynomial ring quotient an ideal 
#F[C_N] = F[x]/(x^N-1)
#and factor x^N-1, and use the Chinese remainder theorem and Bezout's identity for the isomorphism.
#Similarly, we can write down a presentation of S_N, <\sigma, \tau | \sigma^n = 1, \tau^2 = 1, ...>,
#with just two generators and then the group algebra is a quotient of a free algebra on two generators,
#F[S_N] = F<x,y>/ (x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋)
#We still have the Chinese remainder theorem, and there algorthims to factor non-commutative polynomials. 
#It turns out this is equivalent to using central orthogonal idempotents.
#Question: Do we gain anything with this approach?

In [5]:
#Let I = I_1 \cap \ldots \cap I_k be the intersection of pairwise coprime two-sided ideals, 
#i.e. there exists i+j = 1 in each distinct pair I_i, I_j.
#Let phi: F<x,y>/I --> F<x,y>/I_1 \times \ldots \times F<x,y>/I_k
#Let f_i = (0,0, \ldots , 1 , \ldots 0, 0), a 1 in th i-th component and 0's elsewhere.
#The e_i = \phi^-1(f_i). I_i = F<x,y>(1-e_i).
#In this case we have the idempotents e_i = e_i(x,y), so we can construct the ideals I_i = F<x,y>(1-e_i). 
#So really we just need to compute 1-e_i(x,y), so convert the group elements their words in x, y.

In [6]:
#CONSTRUCT EXPLICIT ISOMORPHISM 
#obtain map from quotient of free group to symmetric group using GAP library
#[a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)]
#NOTE: there are many isomorphisms, and a different one is returned every time
def symmetric_group_gens(n):
    f=gap.FreeGroup(2)
    relations = gap([f.1^2, f.2^n, (f.1*f.2)^(n-1), (f.1*f.2^(-1)*f.1*f.2)^3] + [(f.1*f.2^(-j)*f.1*f.2^j)^2 for j in range(2,floor(n/2)+1)])
    g = f / relations
    Sn = gap.SymmetricGroup(n)
    isom = gap.IsomorphismGroups(g,Sn)
    im_g = gap.Image(isom)
    sym_gens = gap.GeneratorsOfGroup(im_g)
    return list(sym_gens)

In [7]:
#map from symmetric group algebra to quotient of free group algebra
#we must use the standard symmetric group in order to solve the word problem
def SGA_to_FGA_quotient(v):
    #solve word problem for each basis element
    FGA_convert = 0
    sym_gens = symmetric_group_gens(n)
    for pair in list(v):
        coeff = FGA(pair[1])
        sigma = SymmetricGroup(n)(pair[0])
        gens = [SymmetricGroup(n)(sym_gens[0]), SymmetricGroup(n)(sym_gens[1])]
        word_gens, word_sym = sigma.word_problem(gens,display=False)
        #handle identity case separately
        if word_gens == "<identity ...>":
            replace_with_free_group_gens = "1"
        else:
            #map word in generators to free group quotient
            replace_with_free_group_gens = word_gens.replace("x1","a").replace("x2","b")
        word_to_free_group_quotient = sage_eval(replace_with_free_group_gens,locals={'a':G.gens()[0],'b':G.gens()[1]})
        FGA_convert += coeff*FGA(word_to_free_group_quotient)
    return FGA_convert

In [8]:
#function to reduce elements of the free group algebra quotient using group relations
def reduce(v):
    k = G.rewriting_system()
    k.make_confluent()
    return sum(FGA(item[1])*FGA(k.reduce(item[0])) for item in list(v))

In [31]:
#a short presentation for S_n
#{x,y|x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋}
n=4
F.<a,b> = FreeGroup()
R = [a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)]
G = F / R
print(G.is_isomorphic(SymmetricGroup(n)))
print(G.structure_description())

True
S4


In [10]:
#we can easily and quickly compute idempotents for the symmetric group algebra
#for each group element in idempotents, find expression as a word in x,y
p=3;
SGA_GFp_n = SymmetricGroupAlgebra(GF(p),n)
idems = SGA_GFp_n.central_orthogonal_idempotents()
idems[2]*idems[2] == idems[2]

True

In [11]:
#can use free group algebra corresponding to G
R = GF(p)
FGA = GroupAlgebra(G, R); FGA

Algebra of Finitely presented group < a, b | a^2, b^4, (a*b)^3, (a*b^-1*a*b)^3, (a*b^-2*a*b^2)^2 > over Finite Field of size 3

In [12]:
print(idems[0])
poly1 = SGA_to_FGA_quotient(idems[0]); reduce(poly1)

2*[1, 2, 4, 3] + 2*[1, 3, 2, 4] + 2*[1, 4, 3, 2] + 2*[2, 1, 3, 4] + [2, 1, 4, 3] + [2, 3, 4, 1] + [2, 4, 1, 3] + [3, 1, 4, 2] + 2*[3, 2, 1, 4] + [3, 4, 1, 2] + [3, 4, 2, 1] + [4, 1, 2, 3] + 2*[4, 2, 3, 1] + [4, 3, 1, 2] + [4, 3, 2, 1]


2*a*b*a*b^-1*a + a*b*a*b^-1*a*b + b^2 + 2*a + b + 2*a*b^2*a*b^-1 + a*b^2*a + 2*b*a*b^-1 + 2*b*a*b^-1*a*b + a*b^2 + b^-1 + a*b*a + a*b^-1*a + b^2*a + 2*b^-1*a*b

In [13]:
print(idems[1])
poly2 = SGA_to_FGA_quotient(idems[1]); reduce(poly2)

[1, 2, 4, 3] + [1, 3, 2, 4] + [1, 4, 3, 2] + [2, 1, 3, 4] + [2, 1, 4, 3] + 2*[2, 3, 4, 1] + 2*[2, 4, 1, 3] + 2*[3, 1, 4, 2] + [3, 2, 1, 4] + [3, 4, 1, 2] + 2*[3, 4, 2, 1] + 2*[4, 1, 2, 3] + [4, 2, 3, 1] + 2*[4, 3, 1, 2] + [4, 3, 2, 1]


a*b*a*b^-1*a + a*b*a*b^-1*a*b + b^2 + a + 2*b + a*b^2*a*b^-1 + a*b^2*a + b*a*b^-1 + b*a*b^-1*a*b + 2*a*b^2 + 2*b^-1 + 2*a*b*a + 2*a*b^-1*a + 2*b^2*a + b^-1*a*b

In [14]:
print(idems[2])
poly3 = SGA_to_FGA_quotient(idems[2]); reduce(poly3)

[1, 2, 3, 4] + [2, 1, 4, 3] + [3, 4, 1, 2] + [4, 3, 2, 1]


1 + a*b*a*b^-1*a*b + b^2 + a*b^2*a

In [15]:
idems[2]

[1, 2, 3, 4] + [2, 1, 4, 3] + [3, 4, 1, 2] + [4, 3, 2, 1]

In [16]:
#the polynomials are still idempotent even when written in terms of generators and relations
reduce(poly3) == reduce(poly3*poly3)

True

In [18]:
#could use Groebner bases for non-commutative polynomials to extract information from idempotent polynomials
#one could use new variables c=a^-1 and d=b^-1 to get rid of the inverses
#then one is factoring a polynomial of four variables
#GAP has a package GBNP, https://www.gap-system.org/Packages/gbnp.html
#we can call Gap from Sage
#need to install GBNP package
#SAGE_ROOT = "/Applications/SageMath-10-3.app/Contents/Frameworks/sage.framework/Versions/10.3"
#GAP_PKG = "local/lib/gap/pkg"

In [28]:
gap('LoadPackage("gbnp");')

true

In [30]:
#should yield 3ab - ba
#gap('PrintNP([[[1,2],[2,1]],[3,-1]]);')
#can only run one GAP command at a time

In [86]:
#FUNDCTION: encodes relations in ["monomial list", "coeff list"] format
#INPUT: relations of group
#OUTPUT: encoded multivariate polynomial in format for input to GBNP in GAP
#R = [a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)]
#NOTE: the zero polynomial is represented by 0 <--> [[],[]] and 1 <--> [[[]],[1]]
#we are encoding the relation a^2=1 from the group as a^2-1=0 in the group algebra
def encode_relations(R, var_index = {'a':1,'b':2,'c':3,'d':4}):
    encoded_relations = []
    for r in R: #iterate over relations
        nc_poly = [[[]],[-1]] #initialize to polynomial -1
        coeff = 1 #the coefficient of the monomial is 1
        monomial = [] #create list of monomial indices
        for item in r.syllables(): #iterate over syllables, e.g. (a,2) <--> a^2
            grp_elem = item[0] #get the group element
            pow = item[1] #get the power the group element is being raised to
            idx = var_index[str(grp_elem)]+2*(pow < 0) #get the index of the variable for the noncommutative polynomial
            for i in range(abs(pow)): #append to monomial list for the number of times the group element occurs
                monomial.append(idx)
        nc_poly[0].append(monomial)
        nc_poly[1].append(coeff)
        encoded_relations.append(nc_poly)
    return encoded_relations

In [87]:
encode_relations(R)

[[[[], [1, 1]], [-1, 1]],
 [[[], [2, 2, 2, 2]], [-1, 1]],
 [[[], [1, 2, 1, 2, 1, 2]], [-1, 1]],
 [[[], [1, 4, 1, 2, 1, 4, 1, 2, 1, 4, 1, 2]], [-1, 1]],
 [[[], [1, 4, 4, 1, 2, 2, 1, 4, 4, 1, 2, 2]], [-1, 1]]]

In [159]:
#encode relations corresponding to a*a^-1 = b*b^-1 = 1, i.e. ac=ca=1, bd=db=1
def inverse_relations():
    return [[[[],[1,3]],[-1,1]],[[[],[3,1]],[-1,1]],[[[],[2,4]],[-1,1]],[[[],[4,2]],[-1,1]]]

In [160]:
inverse_relations()

[[[[], [1, 3]], [-1, 1]],
 [[[], [3, 1]], [-1, 1]],
 [[[], [2, 4]], [-1, 1]],
 [[[], [4, 2]], [-1, 1]]]

In [81]:
#relations for idempotents
def idempotent_nc_poly(idem_poly):
    nc_poly = [[],[]] #initialize to zero polynomial
    #need to look at 1-e_i by Chinese remainder theorem
    for pair in list(1 - idem_poly):
        coeff = pair[1]
        grp_monomial = pair[0]
        monomial = [] #create list of monomial indices
        for item in grp_monomial.syllables(): #iterate over syllables, e.g. (a,2) <--> a^2
            grp_elem = item[0] #get the group element
            pow = item[1] #get the power the group element is being raised to
            idx = var_index[str(grp_elem)]+2*(pow < 0) #get the index of the variable for the noncommutative polynomial
            for i in range(abs(pow)): #append to monomial list for the number of times the group element occurs
                monomial.append(idx)
        nc_poly[0].append(monomial)
        nc_poly[1].append(coeff)
    return nc_poly

In [163]:
#get a list of all relations and ideals as noncommutative polynomials
#we just want to choose the field for the Groebner basis calculation
#can try multiplying by One(GF(p)) in GAP
def all_nc_polynomial_relations():
    all_nc_polys = []
    for idem in idems:
        idem_poly = reduce(SGA_to_FGA_quotient(idem))
        all_nc_polys.append(idempotent_nc_poly(idem_poly))
    all_nc_polys += inverse_relations()
    all_nc_polys += encode_relations(R)
    #format for GAP. convert to GF(p) by multiplying by 1 in the field. format array as string.
    #return all_nc_polys
    return ','.join([ "[" + str(poly[0]) + "," + f"One(GF({p}))*" + str(poly[1]) + "]" for poly in all_nc_polys])

In [166]:
all_nc_polynomial_relations()

'[[[], [1], [1, 2, 2, 1, 4], [4, 1, 2], [2, 1, 4, 1, 2], [2, 1, 4], [1, 2, 1, 4, 1], [1, 2, 1, 4, 1, 2], [1, 2, 2, 1], [2, 2], [1, 4, 1], [4], [2], [2, 2, 1], [1, 2, 1], [1, 2, 2]],One(GF(3))*[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]],[[[], [2, 1, 4, 1, 2], [1, 2, 2, 1, 4], [2, 1, 4], [1], [4, 1, 2], [1, 2, 1, 4, 1], [1, 2, 1, 4, 1, 2], [1, 2, 2, 1], [2, 2], [1, 2, 1], [4], [2], [1, 2, 2], [1, 4, 1], [2, 2, 1]],One(GF(3))*[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]],[[[1, 2, 2, 1], [2, 2], [1, 2, 1, 4, 1, 2]],One(GF(3))*[2, 2, 2]],[[[], [1, 3]],One(GF(3))*[-1, 1]],[[[], [3, 1]],One(GF(3))*[-1, 1]],[[[], [2, 4]],One(GF(3))*[-1, 1]],[[[], [4, 2]],One(GF(3))*[-1, 1]],[[[], [1, 1]],One(GF(3))*[-1, 1]],[[[], [2, 2, 2, 2]],One(GF(3))*[-1, 1]],[[[], [1, 2, 1, 2, 1, 2]],One(GF(3))*[-1, 1]],[[[], [1, 4, 1, 2, 1, 4, 1, 2, 1, 4, 1, 2]],One(GF(3))*[-1, 1]],[[[], [1, 4, 4, 1, 2, 2, 1, 4, 4, 1, 2, 2]],One(GF(3))*[-1, 1]]'

In [165]:
#running the Groebner basis algorithm results in the trivial constant polynomial 1